In [1]:
import os
import xml.etree.cElementTree as et
from bs4 import BeautifulSoup
import pandas as pd
import re
import sys

In [2]:
#Data structure reference
os.chdir('./至106年使用執照')
tree = et.parse("103.xml")
root = tree.getroot()
skip_cols = ['設計人','監造人','承造人','雜項工作物','適用法令概要','全注意事項','變更概要']

for child in root:
    child_dict = {}
    print(child.attrib['號碼'])
    for element in child:
        if element.tag in skip_cols:
            continue
        if element.text is not None:
            print(element.tag,':',element.text)
        else:
            print(element.tag,':',element.text)
            for sub_el in element:
                print(' ',sub_el.tag, ':', sub_el.text)
    print('\t')
    break

103使字第0001號
發照日期 : 1030103
原建照 : 098建字第0379號
建造類別 : 修建
構造種類 : RC造
使用分區 : 住2
建物資訊 : None
  幢數 : 1
  棟數 : 1
  地上層數 : 5
  地下層數 : 2
  戶數 : 5
建物面積 : None
  騎樓基地面積 : 0
  其他基地面積 : 220
  建築面積 : 70.39
  法定空地面積 : 143
  地上避難面積 : 0
  地下避難面積 : 0
簷高 : None
建物高度 : 17.5
工程造價 : 4337518
竣工日期 : 1020205
開工日期 : 0990319
全建築地點 : None
  建築地點 : 臺北市內湖區大湖山莊街21巷14號2樓
  建築地點 : 臺北市內湖區大湖山莊街21巷14號3樓
  建築地點 : 臺北市內湖區大湖山莊街21巷14號4樓
  建築地點 : 臺北市內湖區大湖山莊街21巷14號5樓
  建築地點 : 臺北市內湖區大湖山莊街21巷14號
全地段地號 : None
  地段地號 : 臺北市內湖區康寧段一小段0186-0000號
建築概要 : None
  樓層 : 獨棟棟 地下1樓,面積:92.27㎡,高度:3.55M,用途:停車空間，機房
  樓層 : 獨棟棟 地下2樓,面積:58.99㎡,高度:3.45M,用途:機車停車場，水箱
  樓層 : 獨棟棟 地上1樓,面積:64.87㎡,高度:3.4M,用途:(H2)集合住宅
  樓層 : 獨棟棟 地上2樓,面積:69.87㎡,高度:3.25M,用途:(H2)集合住宅
  樓層 : 獨棟棟 地上3樓,面積:69.87㎡,高度:3.25M,用途:(H2)集合住宅
  樓層 : 獨棟棟 地上4樓,面積:69.87㎡,高度:3.25M,用途:(H2)集合住宅
  樓層 : 獨棟棟 地上5樓,面積:54.7㎡,高度:3.25M,用途:(H2)集合住宅
  樓層 : 獨棟棟 突出物1樓,面積:24.98㎡,高度:3.2M,用途:梯間，機房
  樓層 : 獨棟棟 突出物2樓,面積:24.98㎡,高度:2.9M,用途:梯間，水箱，機房
  樓層 : 獨棟棟 突出物3樓,面積:24.98㎡,高度:2.9M,用途:梯間，機房
全停車空間 : None
  停車空間 : 設置類別

In [3]:
#0 get raw_data from xml
def get_raw():
    #os.chdir('./至106年使用執照')
    data = []
    skip_cols = ['設計人','全設計人','監造人','全監造人','全承造人','承造人',
                 '雜項工作物','適用法令概要','全注意事項','變更概要','全雜項工作物','全起造人','全監造人']

    for filename in os.listdir(os.getcwd()):   
        pattern = '1[0-9][0-9]'    
        if re.search(pattern, filename):
            sys.stdout.write('\nget {}'.format(filename))
            tree = et.parse(filename)
            root = tree.getroot()

            for child in root:
                child_dict = {}
                child_dict['licence_num'] = child.attrib['號碼']
                for element in child:
                    if element.tag in skip_cols:
                        continue

                    if element.text is not None:
                        child_dict[element.tag] = element.text

                    else:
                        num = 2
                        for sub_el in element:  
                            if sub_el.tag in child_dict:
                                new_key = sub_el.tag + str(num)
                                child_dict[new_key] = sub_el.text
                                num += 1
                            else:
                                child_dict[sub_el.tag] = sub_el.text

                data.append(child_dict)
    print('\ncreateing pandas dataframe...')
    df_raw = pd.DataFrame(data)
    print('\ndone')
    return df_raw

def get_sub_frame(df_raw, filter_text):
    output_frame = df_raw.filter(regex=filter_text, axis=1)
    df_licence_num = df_raw.filter(items=['licence_num'])
    output_frame = pd.concat([output_frame,df_licence_num], axis=1)
    
    return output_frame

#unpivot
def melt_frame(melt_df,id_vars):
    melt_cols = list(melt_df.columns[:-1])
    output_frame = pd.melt(melt_df, id_vars=[id_vars], value_vars=melt_cols)
    
    return output_frame



#1-1 floor sub_function
def get_area_num(string):
    try:
        if type(string) != float:
            loc_first_unit = string.find('㎡')
            loc_area_char = string.find('面積')
            if loc_area_char == -1:
                result = 0.0
                return result
            else:
                string_refined = string[loc_area_char:loc_first_unit+1]
                result = re.search('面積(.*)㎡',string_refined)
            if result is not None:
                result = float(result.group(1).replace(':',''))
            else:
                result = string
            return result
    except TypeError:
        print(string)
        pass
    except ValueError:
        print(string)
        pass
    
#1-2 floor sub_function   
def get_area_usage(string):
    if type(string) != float:
        if '使字第' not in string: 
            if '用途' in string:
                loc_unit = string.find('用途:')
                string_refined = string[loc_unit:].replace('用途:','')
                result = string_refined
                return result
            else:
                return '未標明用途:'+string
            
        else:
            return string
        
#1-3 floor sub_function          
def get_area_floorLv(string):
    if type(string) != float:
        if '使字第' not in string: 
            loc_unit = string.find('樓')
            string_refined = string[:loc_unit+1]
            result = string_refined
            return result
        else:
            return string
    
#1 floor function
def get_floor(df_raw):
    sys.stdout.write('\rprocessing floors')
    #1) PROCESSING FLOOR USAGE, AREA AND LEVEL 
    #1.1) extract floor data and licence number
    df_floor_counter = get_sub_frame(df_raw,'樓層')
    
    df_floor_area = df_floor_counter.applymap(lambda x: get_area_num(x))
    df_floor_area_melt = melt_frame(df_floor_area,'licence_num')
    print('\nget_floor: floor_area done')
    
    df_floor_usage = df_floor_counter.applymap(lambda x: get_area_usage(x))
    df_floor_usage_melt = melt_frame(df_floor_usage,'licence_num')
    print('get_floor: floor_usage done')
    
    df_floorLv = df_floor_counter.applymap(lambda x: get_area_floorLv(x))
    df_floorLv_melt = melt_frame(df_floorLv,'licence_num')
    print('get_floor: floor_Lv done')
    
    #1.4) combining 3 dataframe and drop NAN
    print('get_floor: combining...')
    df_floor_result = df_floorLv_melt.join(df_floor_usage_melt, lsuffix='_Lv', rsuffix='_usage')
    df_floor_result = df_floor_result.join(df_floor_area_melt, lsuffix='_re', rsuffix='_area')

    df_floor_result= df_floor_result.drop(['variable_Lv', 'licence_num_usage','variable_usage','licence_num','variable'], axis=1)
    df_floor_result.sort_values(by=['licence_num_Lv'],ascending=True)
    
    df_floor_result = df_floor_result[df_floor_result['value_Lv'].isnull() == False]
    df_floor_result = df_floor_result.sort_values(by=['licence_num_Lv'],ascending=True)
    print('get_floor: combining data done')
    
    
    return df_floor_result

#2 land code
def get_land_code(df_raw):
    sys.stdout.write('\rprocessing land code')
    df_landCode = get_sub_frame(df_raw,'地段地號')
    
    #melt_cols = list(df_landCode.columns)
    #melt_cols.pop(-1)
    
    df_landCode_melt = melt_frame(df_landCode,'licence_num')

    df_landCode_melt = df_landCode_melt.drop(['variable'],axis=1)
    df_landCode_melt = df_landCode_melt[df_landCode_melt['value'].isnull() == False]
    df_landCode_melt = df_landCode_melt.sort_values(by=['licence_num'])
    
    return df_landCode_melt

#3 address
def get_address(df_raw):
    sys.stdout.write('\rprocessing address')
    df_address = get_sub_frame(df_raw,'建築地點')
    df_address_melt = melt_frame(df_address,'licence_num')
    
    df_address_melt = df_address_melt.drop(['variable'],axis=1)
    df_address_melt = df_address_melt[df_address_melt['value'].isnull() == False]
    df_address_melt = df_address_melt.sort_values(by=['licence_num'])
    
    return df_address_melt

#4 overviews
def date_transfor(string):
    try:
        year = str(int(string[:3]) + 1911)
        month = string[3:5]
        day = string[5:]
        date = year + '/' + month +'/' + day
        return date
    except TypeError:
        pass
    
def date_transfor2(string):
    try:
        #print(string)
        if len(str(string)) == 7:
            year = str(int(string[:3]) + 1911)
            month = string[3:5]
            day = string[5:7]
            date = year + '/' + month +'/' + day
            return date
        else:
            year = str(int(string[:2]) + 1911)
            month = string[2:4]
            day = string[4:6]
            date = year + '-' + month +'-' + day
            return date        
    except TypeError:
        return string
    except ValueError:
        pass
    
def get_normal(df_raw):
    drop_cols = ['樓層','地段地號','建築地點','停車空間']
    new_cols = []
    for item in list(df_raw.columns):
        if any(x in item for x in drop_cols):
            pass
        else:
            new_cols.append(item)
    df_normal = df_raw[new_cols].copy()
    df_normal['發照日期'] = df_normal['發照日期'].apply(lambda x: date_transfor(x))
    df_normal['竣工日期'] = df_normal['竣工日期'].apply(lambda x: date_transfor2(x))
    df_normal['開工日期'] = df_normal['開工日期'].apply(lambda x: date_transfor2(x))
    df_normal['發照日期'] = pd.to_datetime(df_normal['發照日期'],errors='coerce')
    df_normal['竣工日期'] = pd.to_datetime(df_normal['竣工日期'],errors='coerce')
    df_normal['開工日期'] = pd.to_datetime(df_normal['開工日期'],errors='coerce')
    
    df_normal['工程造價'] = pd.to_numeric(df_normal['工程造價'], downcast='float')
    df_normal['工程造價'] = df_normal['工程造價']/10000
    df_normal['工程造價'] = df_normal['工程造價'].round(0)

    
    df_normal = df_normal.sort_values(by=['licence_num'],ascending=True)
    return df_normal


In [4]:
#0 get raw data
df_raw = get_raw()


get 101.xml
get 100.xml
get 102.xml
get 103.xml
get 106.xml
get 104.xml
get 105.xml
createing pandas dataframe...

done


In [5]:
df_raw.columns

Index(['licence_num', '使用分區', '停車空間', '停車空間2', '停車空間3', '停車空間4', '停車空間5',
       '停車空間6', '停車空間7', '停車空間8',
       ...
       '樓層96', '樓層97', '樓層98', '樓層99', '法定空地面積', '發照日期', '竣工日期', '簷高', '開工日期',
       '騎樓基地面積'],
      dtype='object', length=1331)

In [6]:
df_raw.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288 entries, 0 to 2287
Columns: 1331 entries, licence_num to 騎樓基地面積
dtypes: object(1331)
memory usage: 104.1 MB


In [7]:
df_raw.sort_values(by='licence_num',ascending=True).head()

,licence_num,使用分區,停車空間,停車空間2,停車空間3,停車空間4,停車空間5,停車空間6,停車空間7,停車空間8,...,樓層96,樓層97,樓層98,樓層99,法定空地面積,發照日期,竣工日期,簷高,開工日期,騎樓基地面積
380,100使字第0001號,住宅區,"設置類別:平面,車位分類:汽車,檢討類別:法定,室內,地下,輛數:2,面積:㎡","設置類別:平面,車位分類:機車,檢討類別:法定,室內,地下,輛數:23,面積:㎡","設置類別:平面,車位分類:汽車,檢討類別:自設,室內,地下,輛數:8,面積:㎡","設置類別:平面,車位分類:,檢討類別:法定,室內,地上,輛數:10,面積:㎡","設置類別:平面,車位分類:,檢討類別:法定,室外,地上,輛數:2,面積:㎡","設置類別:機械,車位分類:汽車,檢討類別:法定,室內,地下,輛數:14,面積:304㎡",NaN,NaN,...,NaN,NaN,NaN,NaN,470.18,1000103,0990831,NaN,0971007,0
381,100使字第0002號,師專用地,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,42915,1000103,0990201,NaN,0980921,0
382,100使字第0003號,保護區,"設置類別:平面,車位分類:汽車,檢討類別:法定,室內,地下,輛數:3,面積:㎡","設置類別:平面,車位分類:機車,檢討類別:法定,室內,地下,輛數:6,面積:㎡","設置類別:平面,車位分類:汽車,檢討類別:自設,室內,地下,輛數:13,面積:㎡","設置類別:平面,車位分類:機車,檢討類別:自設,室內,地下,輛數:5,面積:245.28㎡",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7306.5,1000103,0990819,NaN,0921007,0
383,100使字第0004號,住2,"設置類別:平面,車位分類:汽車,檢討類別:法定,室內,地上,輛數:1,面積:15㎡","設置類別:平面,車位分類:機車,檢討類別:法定,室外,地上,輛數:3,面積:5.94㎡","設置類別:機械,車位分類:汽車,檢討類別:法定,室內,地上,輛數:2,面積:12.1㎡",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,121.55,1000103,0990906,NaN,0980427,0
384,100使字第0005號,機關用地(公共設施用地)(供社會福利設施、區民活動中心等使用),"設置類別:平面,車位分類:汽車,檢討類別:法定,室內,地下,輛數:43,面積:603.75㎡","設置類別:平面,車位分類:裝卸位,檢討類別:法定,室外,地上,輛數:2,面積:30㎡","設置類別:平面,車位分類:機車,檢討類別:法定,室內,地下,輛數:179,面積:354.42㎡",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1554,1000104,0990906,NaN,0971027,0


In [8]:
#1 FLOOR COMPLETE
df_floor = get_floor(df_raw)
df_floor.head(10)

processing floors
get_floor: floor_area done
get_floor: floor_usage done
get_floor: floor_Lv done
get_floor: combining...
get_floor: combining data done


,licence_num_Lv,value_Lv,value_usage,value
53004,100使字第0001號,1棟 地上9樓,集合住宅(H2),135.91
238332,100使字第0001號,1棟 地上3樓,集合住宅(H2),238.43
288668,100使字第0001號,1棟 地上5樓,集合住宅(H2),238.43
78172,100使字第0001號,1棟 突出物1樓,電梯間.屋頂平台,24.72
2668,100使字第0001號,1棟 地上7樓,集合住宅(H2),238.43
380,100使字第0001號,1棟 地下1樓,防空避難室兼停車空間,468.00
187996,100使字第0001號,1棟 地上1樓,"管委會辦公室,日常用品零售業(G3)自行車停車空間\n",234.45
103340,100使字第0001號,1棟 突出物2樓,電梯機械室,26.27
213164,100使字第0001號,1棟 地上2樓,集合住宅(H2),238.43
162828,100使字第0001號,1棟 地下3樓,停車空間.消防機房,468.00


In [9]:
#land code
df_landCode = get_land_code(df_raw)
df_landCode.head(10)

processing land code

,licence_num,value
380,100使字第0001號,臺北市內湖區潭美段五小段0071-0000號
762285,100使字第0002號,臺北市大安區學府段一小段0392-0000號
963629,100使字第0002號,臺北市大安區學府段一小段0566-0000號
938461,100使字第0002號,臺北市大安區學府段一小段0393-0000號
988797,100使字第0002號,臺北市大安區學府段一小段0569-0000號
381,100使字第0002號,臺北市大安區學府段一小段0591-0000號
1013965,100使字第0002號,臺北市大安區學府段一小段0573-0000號
508317,100使字第0002號,臺北市大安區學府段一小段0391-0000號
254349,100使字第0002號,臺北市大安區學府段一小段0592-0000號
382,100使字第0003號,臺北市信義區永春段三小段0127-0000號


In [10]:
df_address = get_address(df_raw)
df_address.head()

processing address

,licence_num,value
1270220,100使字第0001號,臺北市內湖區行善路333巷66號7樓
380,100使字第0001號,臺北市內湖區行善路333巷66號2樓
128508,100使字第0001號,臺北市內湖區行善路333巷68號8樓
153676,100使字第0001號,臺北市內湖區行善路333巷68號
1496732,100使字第0001號,臺北市內湖區行善路333巷66號8樓


In [11]:
df_normal = get_normal(df_raw)
df_normal.head()

,licence_num,使用分區,其他基地面積,原建照,地上層數,地上避難面積,地下層數,地下避難面積,工程造價,幢數,...,建造類別,戶數,棟數,構造種類,法定空地面積,發照日期,竣工日期,簷高,開工日期,騎樓基地面積
380,100使字第0001號,住宅區,723.36,097建字第0032號,9,0,3,393.6,3932.0,1,...,新建,16,1,RC造,470.18,2011-01-03,2010-08-31,NaN,2008-10-07,0
381,100使字第0002號,師專用地,71525,098建字第0203號,5,0,0,0,107.0,1,...,增建,1,1,鋼骨造,42915,2011-01-03,2010-02-01,NaN,2009-09-21,0
382,100使字第0003號,保護區,9742,092建字第0019號,1,NaN,1,NaN,1744.0,1,...,新建,1,1,RC造,7306.5,2011-01-03,2010-08-19,NaN,2003-10-07,0
383,100使字第0004號,住2,187,098建字第0125號,5,0,0,0,261.0,1,...,新建,1,1,RC造,121.55,2011-01-03,2010-09-06,NaN,2009-04-27,0
384,100使字第0005號,機關用地(公共設施用地)(供社會福利設施、區民活動中心等使用),2595,097建字第0282號,10,0,2,1834.8,14702.0,0,...,新建,13,1,RC造,1554,2011-01-04,2010-09-06,NaN,2008-10-27,0


In [12]:
df_normal.to_csv('../result/usage_normal.csv', sep=',', encoding='utf-8', index=False)
df_floor.to_csv('../result/usage_floor.csv', sep=',', encoding='utf-8', index=False)
df_landCode.to_csv('../result/usage_lanCode.csv', sep=',', encoding='utf-8', index=False)
df_address.to_csv('../result/usage_address.csv', sep=',', encoding='utf-8', index=False)